In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

from ipywidgets import Layout, FloatSlider, Checkbox, Text, widgets, IntSlider, Dropdown

from simple_analysis_scripts.potential_analysis.analyze_potential import *

import io
import matplotlib.pyplot as plt
from PIL import Image
import win32clipboard

def copy_figure_to_clipboard(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=300, bbox_inches="tight")
    buf.seek(0)

    image = Image.open(buf)
    output = io.BytesIO()
    image.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]  # BMP header removed

    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

# Single system analysis:

In [3]:
def f(n_actual, dn, n_rays, unconcentricity, phi_max, desired_focus, T_c, diameter, plot, lens_type, copy_input_parameters, copy_cavity_parameters, eval_box, copy_image):

    if copy_input_parameters:
        copy_parameters_func(locals())

    n_actual, n_design, T_c, back_focal_length, R_1, R_2, R_2_signed, diameter = known_lenses_generator(lens_type, dn)
    defocus = choose_source_position_for_desired_focus_analytic(desired_focus=desired_focus, T_c=T_c, n_design=n_design, diameter=diameter, back_focal_length=back_focal_length, R_1=R_1, R_2=R_2_signed,)
    optical_system, optical_axis = generate_one_lens_optical_system(R_1=R_1, R_2=R_2_signed, back_focal_length=back_focal_length, defocus=defocus, T_c=T_c, n_design=n_design, diameter=diameter, n_actual=n_actual, )
    rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max, diameter=diameter, back_focal_length=back_focal_length)
    results_dict = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=unconcentricity, print_tests=False)

    if plot:
        # plt.close('all')
        fig, ax = plot_results(results_dict, far_away_plane=True, unconcentricity=unconcentricity)
        center = results_dict["center_of_curvature"]
        plt.suptitle(
            f"aspheric={aspheric}, desired_focus = {desired_focus:.3e}m, n_design: {n_design:.3f}, n_actual: {n_actual:.3f}, Lens focal length: {back_focal_length * 1e3:.1f} mm, Defocus: z_lens -> z_lens + {defocus * 1e3:.1f} mm, T_c: {T_c * 1e3:.1f} mm, Diameter: {diameter * 1e3:.2f} mm"
        )
        plt.show()

    if copy_cavity_parameters:
        pyperclip.copy(results_dict['cavity'].formatted_textual_params)

    if copy_image:
        copy_figure_to_clipboard(fig)
    
    if eval_box != '':
        try:
            exec(f"print({eval_box})")
        except (NameError, AttributeError) as e:
            print(f'invalid expression: {e}')

# rest of parameters
n_actual = 1.8
dn = 0
n_rays = 30
unconcentricity = 30e-4  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.25
desired_focus = 200e-3
T_c = 4.35e-3
diameter = 12.7e-3
plot = True
aspheric = True
lens_type = 'aspheric - lab'

widgets.interact(
    f,
    n_actual=FloatSlider(value=n_actual, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    dn=FloatSlider(value=dn, min=-0.2, max=0.2, step=0.0001, description='dn', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    unconcentricity=FloatSlider(value=unconcentricity, min=0.0, max=100e-3, step=1e-6, description='unconcentricity (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.01, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    desired_focus=FloatSlider(value=desired_focus, min=1e-3, max=1.0, step=1e-4, description='desired focus (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c=FloatSlider(value=T_c, min=0.5e-3, max=10e-3, step=1e-5, description='T_c (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    diameter=FloatSlider(value=diameter, min=1e-3, max=25e-3, step=1e-5, description='diameter (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    plot=Checkbox(value=plot, description='Plot results', layout=Layout(width='300px'), style={'description_width': 'initial'}),
    lens_type=Dropdown(options=[("Existing Aspheric", 'aspheric - lab'), ('Avantier', 'avantier'), ('Spherical - f, n like labs aspheric', 'spherical - like labs aspheric'), ('Aspherical - f, n like Avantier', 'aspheric - like avantier')], value=lens_type, description="Collimation mode", style={'description_width': 'initial'}),
    copy_input_parameters=Checkbox(value=False, description='Copy input parameters', style={'description_width': 'initial'}),
    copy_cavity_parameters=Checkbox(value=False, description='Copy cavity parameters', style={'description_width': 'initial'}),
    eval_box=Text(value='', placeholder='Type a Python expression to print (e.g., cavity.arms[0].length)', description='Evaluate:', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),

);

interactive(children=(FloatSlider(value=1.8, description='n actual', layout=Layout(width='1500px'), max=2.0, m…

# Multiple Unconcentricities analysis:

In [6]:
def f(maximal_unconcentricitiy, dn, n_rays, phi_max, desired_focus, lens_type, copy_input_parameters, copy_image):  # , eval_box
    if copy_input_parameters:
        copy_parameters_func(locals())

    n_actual, n_design, T_c, back_focal_length, R_1, R_2, R_2_signed, diameter = known_lenses_generator(lens_type, dn)
    defocus = choose_source_position_for_desired_focus_analytic(
    desired_focus=desired_focus,
    T_c=T_c,
    n_design=n_design,
    diameter=diameter,
    back_focal_length=back_focal_length,
    R_1=R_1,
    R_2=R_2_signed,
    )
    unconcentricities = np.linspace(maximal_unconcentricitiy, 0.1e-3, 30)
    paraxial_spot_sizes = np.zeros_like(unconcentricities)
    spot_size_boundaries = np.zeros_like(unconcentricities)
    paraxial_NAs = np.zeros_like(unconcentricities)
    left_NAs = np.zeros_like(unconcentricities)
    for i, u in enumerate(unconcentricities):
        defocus = choose_source_position_for_desired_focus_analytic(desired_focus=desired_focus, T_c=T_c, n_design=n_design, diameter=diameter, back_focal_length=back_focal_length, R_1=R_1, R_2=R_2_signed,)
        optical_system, optical_axis = generate_one_lens_optical_system(R_1=R_1, R_2=R_2_signed, back_focal_length=back_focal_length, defocus=defocus, T_c=T_c, n_design=n_design, diameter=diameter, n_actual=n_actual, )
        rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max, diameter=diameter, back_focal_length=back_focal_length)
        results_dict = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=u, print_tests=False)
        paraxial_spot_sizes[i] = results_dict["spot_size_paraxial"]
        paraxial_NAs[i] = results_dict["NA_paraxial"]
        left_NAs[i] = results_dict["cavity"].arms[0].mode_parameters.NA[0]
        try:
            spot_size_boundaries[i] = np.abs(results_dict["zero_derivative_points"])
        except TypeError:
            spot_size_boundaries[i] = np.nan  # If zero_derivative_points is None
    # %%
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(unconcentricities * 1e3, paraxial_spot_sizes * 1e3, label="Paraxial spot size", color="blue")
    ax.scatter(
        unconcentricities * 1e3, spot_size_boundaries * 1e3, label="Boundary of 2nd vs 4th order dominance", color="red"
    )
    ax.set_xlabel("Unconcentricity (mm)")
    ax.set_ylabel("Spot size / boundary (mm)")
    ax.set_title(f"paraxial spot size vs. aberrations limit\nLens type:={lens_type}, Concentric long arm length: {2 * desired_focus:.2e}")
    ax.grid()

    # twin y-axis for paraxial NAs
    ax2 = ax.twinx()
    ax2.plot(unconcentricities * 1e3, paraxial_NAs, label="Right NA", color="orange", linestyle="--")
    ax2.plot(unconcentricities * 1e3, left_NAs, label="Left NA", color="green", linestyle="--")
    ax2.set_ylabel("Paraxial NA (unitless)")

    # combined legend
    handles1, labels1 = ax.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(handles1 + handles2, labels1 + labels2, loc="best")

    if copy_image:
        copy_figure_to_clipboard(fig)
    plt.show()
    # if eval_box != '':
    #     try:
    #         exec(f"print({eval_box})")
    #     except (NameError, AttributeError) as e:
    #         print(f'invalid expression: {e}')

# rest of parameters
n_actual = 1.8
dn = 0
n_rays = 500
unconcentricity = 30e-4  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.25
desired_focus = 200e-3
T_c = 4.35e-3
diameter = 12.7e-3
plot = True
aspheric = True
lens_type ='aspheric - lab'

widgets.interact(
    f,
    n_actual=FloatSlider(value=n_actual, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    maximal_unconcentricitiy=FloatSlider(value=10e-3, min=1e-3, max=200e-3, step=0.0001, description='Maximal Unconcentricity', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    dn=FloatSlider(value=dn, min=-0.2, max=0.2, step=0.0001, description='dn', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.01, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    desired_focus=FloatSlider(value=desired_focus, min=1e-3, max=1.0, step=1e-4, description='desired focus (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c=FloatSlider(value=T_c, min=0.5e-3, max=10e-3, step=1e-5, description='T_c (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    diameter=FloatSlider(value=diameter, min=1e-3, max=25e-3, step=1e-5, description='diameter (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    lens_type=Dropdown(options=[("Existing Aspheric", 'aspheric - lab'), ('Avantier', 'avantier'), ('Spherical - f, n like labs aspheric', 'spherical - like labs aspheric'), ('Aspherical - f, n like Avantier', 'aspheric - like avantier')], value=lens_type, description="Collimation mode", style={'description_width': 'initial'}),
    copy_input_parameters=Checkbox(value=False, description='Copy input parameters', style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),
    # eval_box=Text(value='', placeholder='Type a Python expression to print (e.g., cavity.arms[0].length)', description='Evaluate:', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
);

interactive(children=(FloatSlider(value=0.01, description='Maximal Unconcentricity', layout=Layout(width='1500…

# Two lenses cavity, single unconcentricity:

In [8]:
def f(
        n_actual_aspheric,
        n_rays,
        copy_image,
        unconcentricity,
        phi_max,
        defocus,
        back_focal_length_aspheric,
        T_c_aspheric,
        n_design_aspheric,
        n_design_spherical,
        n_actual_spherical,
        T_c_spherical,
        f_spherical,
        diameter,
):
    optical_system = generate_two_lenses_optical_system(defocus=defocus, back_focal_length_aspheric=back_focal_length_aspheric, T_c_aspheric=T_c_aspheric, n_design_aspheric=n_design_aspheric, n_actual_aspheric=n_actual_aspheric, n_design_spherical=n_design_spherical, n_actual_spherical=n_actual_spherical, T_c_spherical=T_c_spherical, f_spherical=f_spherical, diameter=diameter,)
    rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max)
    results_dict = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=unconcentricity, print_tests=False, )
    fig, ax = plot_results(results_dict=results_dict, far_away_plane=True, unconcentricity=unconcentricity, potential_x_axis_angles=False, )

    if copy_image:
        copy_figure_to_clipboard(fig)

# rest of parameters
n_rays = 50
unconcentricity = 1e-3  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.3
defocus = 0.0029887489470528557
back_focal_length_aspheric = 20e-3
T_c_aspheric = 4.35e-3
n_design_aspheric = 1.45
n_actual_aspheric = 1.45
n_design_spherical = 1.45
n_actual_spherical = 1.45
T_c_spherical = 4.35e-3
f_spherical = 100e-3
diameter = 12.7e-3

widgets.interact(
    f,
    n_actual_aspheric=FloatSlider(value=n_actual_aspheric, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),
    unconcentricity=FloatSlider(value=unconcentricity, min=-0.02, max=0.02, step=1e-5, description='unconcentricity', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    phi_max=FloatSlider(value=phi_max, min=0.0, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    defocus=FloatSlider(value=defocus, min=-0.01, max=0.01, step=1e-6, description='defocus', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    back_focal_length_aspheric=FloatSlider(value=back_focal_length_aspheric, min=1e-3, max=0.1, step=1e-4, description='back focal length (aspheric)', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    T_c_aspheric=FloatSlider(value=T_c_aspheric, min=0.0, max=0.02, step=1e-5, description='T_c aspheric', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    n_design_aspheric=FloatSlider(value=n_design_aspheric, min=1.0, max=2.0, step=0.0001, description='n design (aspheric)', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    n_design_spherical=FloatSlider(value=n_design_spherical, min=1.0, max=2.0, step=0.0001, description='n design (spherical)', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    n_actual_spherical=FloatSlider(value=n_actual_spherical, min=1.0, max=2.0, step=0.0001, description='n actual (spherical)', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    T_c_spherical=FloatSlider(value=T_c_spherical, min=0.0, max=0.02, step=1e-5, description='T_c spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    f_spherical=FloatSlider(value=f_spherical, min=1e-3, max=0.5, step=1e-4, description='f spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    diameter=FloatSlider(value=diameter, min=1e-3, max=0.05, step=1e-4, description='diameter', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
);
plt.show()


interactive(children=(FloatSlider(value=1.45, description='n actual', layout=Layout(width='1500px'), max=2.0, …

# Custom lens design:

In [4]:
def f(
        copy_image,
        lens_type,
        defocus,
        unconcentricity,
        n_rays,
        phi_max,
        back_focal_length_aspheric,
        r_spherical,
        polynomial_coefficients_back_2,
        polynomial_coefficients_back_4,
        polynomial_coefficients_back_6,
):
    
    # rest of parameters
    T_c_aspheric = 4.35e-3
    diameter = 12.7e-3
    n_design_aspheric = 1.45
    n_actual_aspheric = 1.45
    
    polynomial_coefficients_back_2 = widget_convenient_exponent(polynomial_coefficients_back_2, scale=0)
    polynomial_coefficients_back_4 = widget_convenient_exponent(polynomial_coefficients_back_4, scale=0)
    polynomial_coefficients_back_6 = widget_convenient_exponent(polynomial_coefficients_back_6, scale=0)

    rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max)
    
    aspheric_flat, aspheric_front = Surface.from_params(
        generate_aspheric_lens_params(back_focal_length=back_focal_length_aspheric, T_c=T_c_aspheric,n=n_design_aspheric, forward_normal=OPTICAL_AXIS, flat_faces_center=back_center, diameter=diameter,polynomial_degree=8, name="aspheric_lens_automatic",)
    )

    spherical_back_alternative = CurvedRefractiveSurface(radius=r_spherical, curvature_sign=CurvatureSigns.convex, outwards_normal=-OPTICAL_AXIS, center=aspheric_flat.center, n_1=1, n_2=1.45, name="spherical_lens_alternative", material_properties=PHYSICAL_SIZES_DICT["material_properties_fused_silica"], thickness=T_c_aspheric / 2, diameter=diameter,)
    
    polynomial_coefficients_back = np.array([0, polynomial_coefficients_back_2, polynomial_coefficients_back_4, polynomial_coefficients_back_6])
    outwards_normal = -OPTICAL_AXIS * np.sign(polynomial_coefficients_back[1]) if polynomial_coefficients_back[1] != 0 else -OPTICAL_AXIS
    if polynomial_coefficients_back[1] < 0:
        polynomial_coefficients_back[1] *= -1
    aspherical_back_alternative = AsphericRefractiveSurface(polynomial_coefficients=polynomial_coefficients_back, center=aspheric_flat.center, n_1=1, n_2=1.45, outwards_normal=outwards_normal,name="aspherical_lens_alternative", material_properties=PHYSICAL_SIZES_DICT["material_properties_fused_silica"], thickness=T_c_aspheric / 2, diameter=diameter, curvature_sign=outwards_normal @ OPTICAL_AXIS)
    
    if lens_type == 'original':
        optical_system = OpticalSystem(surfaces=[aspheric_flat, aspheric_front], t_is_trivial=True, p_is_trivial=True, given_initial_central_line=True, use_paraxial_ray_tracing=False, )
    elif lens_type == 'aspheric back':
        optical_system = OpticalSystem(surfaces=[aspherical_back_alternative, aspheric_front], t_is_trivial=True, p_is_trivial=True, given_initial_central_line=True, use_paraxial_ray_tracing=False, )
    else:
        optical_system = OpticalSystem(surfaces=[spherical_back_alternative, aspheric_front], t_is_trivial=True, p_is_trivial=True, given_initial_central_line=True, use_paraxial_ray_tracing=False, )

    results_dict_spherical_back = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=unconcentricity, print_tests=False)
    fig, ax = plot_results(results_dict=results_dict_spherical_back, far_away_plane=True, unconcentricity=unconcentricity, potential_x_axis_angles=False, rays_labels=["Before lens", "After flat surface", "After spherical surface"])
    
    if copy_image:
        copy_figure_to_clipboard(fig)

plot = True
n_rays = 50
unconcentricity = 1e-3  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.05
OPTICAL_AXIS = RIGHT
back_focal_length_aspheric = 20e-3
defocus = -0.002
back_center = (back_focal_length_aspheric - defocus) * OPTICAL_AXIS
lens_type = 'aspheric back'

widgets.interact(
    f,
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),
    lens_type=Dropdown(options=[("original", 'original'), ('aspheric back', 'aspheric back'), ('spherical back', 'spherical back')], value=lens_type, description="Collimation mode", style={'description_width': 'initial'}),
    defocus=FloatSlider(value=defocus, min=-0.01, max=0.01, step=1e-6, description='defocus', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    unconcentricity=FloatSlider(value=unconcentricity, min=-0.02, max=0.02, step=1e-5, description='unconcentricity', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.0, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    back_focal_length_aspheric=FloatSlider(value=back_focal_length_aspheric, min=1e-3, max=0.1, step=1e-4, description='back focal length (aspheric)', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    r_spherical=FloatSlider(value=200e-3, min=1e-3, max=1.0, step=1e-4, description='radius spherical back', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    polynomial_coefficients_back_2=FloatSlider(value=0.0, min=-3, max=3, step=1e-3, description='polynomial coefficient back 2', layout=Layout(width='1500px'), style={'description_width': 'initial'}, readout_format='.3e'),
    polynomial_coefficients_back_4=FloatSlider(value=0.0, min=-6, max=6, step=1e-3, description='polynomial coefficient back 4', layout=Layout(width='1500px '), style={'description_width': 'initial'}, readout_format='.3e'),
    polynomial_coefficients_back_6=FloatSlider(value=0.0, min=-9, max=9, step=1e-3, description='polynomial coefficient back 6', layout=Layout(width='1500px '), style={'description_width': 'initial'}, readout_format='.3e'),
);



interactive(children=(Checkbox(value=False, description='Copy image', style=CheckboxStyle(description_width='i…

# Negative lens:

In [25]:

OPTICAL_AXIS = RIGHT
def f(
        copy_image,
        copy_input_parameters,
        copy_cavity_parameters,
        lens_type,
        desired_focus,
        unconcentricity,
        n_rays,
        phi_max,
        spherical_radii,
        T_c_spherical,
        n_actual_spherical,
        relative_position_spherical,
):
    if copy_input_parameters:
        copy_parameters_func(locals())
    
    unconcentricity = widget_convenient_exponent(unconcentricity, scale=-4)
    n_actual, n_design, T_c, back_focal_length, R_1, R_2, R_2_signed, diameter = known_lenses_generator(lens_type, 0)
    defocus = choose_source_position_for_desired_focus_analytic(desired_focus=desired_focus, T_c=T_c, n_design=n_design, diameter=diameter, back_focal_length=back_focal_length, R_1=R_1, R_2=R_2_signed, )
    optical_system_initial, optical_axis = generate_one_lens_optical_system(R_1=R_1, R_2=R_2_signed, back_focal_length=back_focal_length, defocus=defocus, T_c=T_c, n_design=n_design, diameter=diameter, n_actual=n_actual,)
    output_ROC = optical_system_initial.output_radius_of_curvature(initial_distance=optical_system_initial.surfaces[0].center[0])
    spherical_back_center = optical_system_initial.surfaces[-1].center + output_ROC * (1 + relative_position_spherical) * OPTICAL_AXIS
    spherical_back = CurvedRefractiveSurface(radius=spherical_radii, center=spherical_back_center, outwards_normal=OPTICAL_AXIS, n_1=1, n_2=n_actual_spherical, name="spherical_lens_concave_back", material_properties=PHYSICAL_SIZES_DICT["material_properties_fused_silica"], thickness=T_c_spherical / 2, diameter=diameter, curvature_sign=CurvatureSigns.concave, )
    spherical_front = CurvedRefractiveSurface(radius=spherical_radii, center=spherical_back_center + T_c_spherical * OPTICAL_AXIS, outwards_normal=-OPTICAL_AXIS, n_1=n_actual_spherical, n_2=1, name="spherical_lens_concave_front", material_properties=PHYSICAL_SIZES_DICT["material_properties_fused_silica"], thickness=T_c_spherical / 2, diameter=diameter, curvature_sign=CurvatureSigns.convex, )
    optical_system = OpticalSystem(surfaces=[*optical_system_initial.surfaces, spherical_back, spherical_front], t_is_trivial=True, p_is_trivial=True, use_paraxial_ray_tracing=False, given_initial_central_line=True, )
    rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max)
    results_dict = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=unconcentricity, end_mirror_ROC=10e-2, print_tests=False, )
    fig, ax = plot_results(results_dict=results_dict, far_away_plane=True, unconcentricity=unconcentricity, potential_x_axis_angles=False, )

    ax[1, 0].set_title(f"large mirror spot size: {results_dict['cavity'].arms[4].mode_parameters_on_surface_1.spot_size[0]:.2e}, long arm NA: {results_dict['cavity'].arms[4].mode_parameters.NA[0]:.3f}\nshort arm NA: {results_dict['cavity'].arms[0].mode_parameters.NA[0]:.3f}, ")
    ax[1, 1].set_title(f"lens type: {lens_type}, relative_position_spherical={relative_position_spherical:.2e}\nunconcentricity={unconcentricity:.2e}, negative lens radius: {spherical_radii:.3e}, n negative lens={n_actual_spherical:.2f}")
    
    if copy_image:
        copy_figure_to_clipboard(fig)

    if copy_cavity_parameters:
        pyperclip.copy(results_dict['cavity'].formatted_textual_params)

    plt.show()


plot = True
n_rays = 50
unconcentricity = 1e-3  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.15
back_focal_length_aspheric = 20e-3
defocus = -0.002
back_center = (back_focal_length_aspheric - defocus) * OPTICAL_AXIS
lens_type = 'aspheric - lab'
desired_focus = 200e-3
spherical_radii = 200e-3
T_c_spherical = 4.35e-3
n_actual_spherical = 1.45
relative_position_spherical = 0.3

widgets.interact(
    f,
    lens_type=Dropdown(options=[("Existing Aspheric", 'aspheric - lab'), ('Avantier', 'avantier'), ('Spherical - f, n like labs aspheric', 'spherical - like labs aspheric'), ('Aspherical - f, n like Avantier', 'aspheric - like avantier')], value=lens_type, description="Lens type", style={'description_width': 'initial'}),
    desired_focus=FloatSlider(value=desired_focus, min=1e-3, max=1.0, step=1e-4, description='desired focus (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    unconcentricity=FloatSlider(value=unconcentricity, min=0.0, max=0.1, step=1e-6, description='unconcentricity (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.01, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    spherical_radii=FloatSlider(value=spherical_radii, min=1e-3, max=2.0, step=1e-4, description='spherical radii (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c_spherical=FloatSlider(value=T_c_spherical, min=0.5e-3, max=10e-3, step=1e-5, description='T_c spherical (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_actual_spherical=FloatSlider(value=n_actual_spherical, min=1.0, max=2.0, step=0.0001, description='n actual spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    relative_position_spherical=FloatSlider(value=relative_position_spherical, min=-1.0, max=3.0, step=0.001, description='relative position spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    copy_input_parameters=Checkbox(value=False, description='Copy input parameters', style={'description_width': 'initial'}),
    copy_cavity_parameters=Checkbox(value=False, description='Copy cavity parameters', style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),

);

interactive(children=(Checkbox(value=False, description='Copy image', style=CheckboxStyle(description_width='i…